<a href="https://colab.research.google.com/github/mkm-world/laduma-analytics/blob/main/zindi_football_baseline_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Zindi Weekendz: Laduma Analytics Football League Winners Prediction Challenge

- **Can you predict the outcome of a football match based on historical data?**

# Downloading all datasets directly from Zindi platform using url:

In [ ]:
import requests, zipfile
# The url and auth_value from the website
myobj = {'auth_token': ''} # fill with your token
data_list=['Test.csv','VariableDefinitions.csv','Train.csv','SampleSubmission.csv']
target_dir=''
base_path='https://api.zindi.africa/v1/competitions/laduma-analytics-football-league-winners-prediction-challenge/files/'
def load_zindi_data(data_list,base_path,target_dir):
  for data in data_list:
      target_path=  target_dir +data
      data_path=base_path+ data
      x = requests.post(data_path, data = myobj,stream=True)
      handle = open(target_path, "wb")
      for chunk in x.iter_content(chunk_size=512):
        if chunk:  # filter out keep-alive new chunks
          handle.write(chunk)
      handle.close()
load_zindi_data(data_list,base_path,target_dir)

#### Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools
from tqdm import tqdm
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

#Load Datasets:

In [ ]:
train = pd.read_csv("Train.csv",parse_dates=['Date'])
test = pd.read_csv("Test.csv",parse_dates=['Date'])

In [ ]:
def preprocess(train,test):
  train['Train']=1
  test['Train']=0
  home_teams=train['Home Team'].unique()
  all_data=pd.concat([train,test])
  for team in home_teams:
    all_data['home_'+team]=0
    all_data.loc[all_data['Home Team']==team,'home_'+team]=1
  away_teams=train['Away Team'].unique()
  for team in away_teams:
    all_data['away_'+team]=0
    all_data.loc[all_data['Away Team']==team,'away_'+team]=1
  train_cols = all_data.columns.difference(['Date', 'Season', 'Match_ID', 'Game_ID','Score','Train','Home Team','Away Team'])
  all_data.fillna(all_data.mean(),inplace=True)
  train = all_data[all_data["Train"] == 1]
  test = all_data[all_data["Train"] == 0]
  return train[train_cols],test[train_cols],train['Score'],test['Score']

In [ ]:
le=LabelEncoder()
train["Score"] = le.fit_transform(train["Score"])
score_mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [ ]:
train.sort_values(by=['Date'],inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
trai,val= train_test_split(train, test_size=0.2,shuffle=False)

In [ ]:
pro_train_X,pro_test_X,pro_train_y,pro_test_y=preprocess(trai,val)

In [ ]:
X,test_data,y,_=preprocess(train,test)

In [ ]:
#importing models
import lightgbm as lgb 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss

In [ ]:
lr=LogisticRegression(C=0.1,max_iter=1000)
lr.fit(pro_train_X,pro_train_y)
#Predictiing on Test Set
y_pred = lr.predict_proba(pro_test_X)
print(log_loss(pro_test_y,y_pred))

1.0714671756358163


In [ ]:
xgb=XGBClassifier(n_estimators=50 , max_depth=4, learning_rate=0.08,colsample_bytree=0.2,
                  random_state=12,subsample=0.8)
xgb.fit(pro_train_X,pro_train_y)
#Predictiing on Test Set
y_pred = xgb.predict_proba(pro_test_X)
print(log_loss(pro_test_y,y_pred))

1.0747329903748428


In [ ]:
clf = RandomForestClassifier(n_estimators=120 , random_state=12,max_depth=2,max_features=0.8)
clf.fit(pro_train_X,pro_train_y)
#Predictiing on Test Set
y_pred = clf.predict_proba(pro_test_X)
print(log_loss(pro_test_y,y_pred))

In [ ]:
lr.fit(X,y)
test_predictions  = lr.predict_proba(test_data)

In [ ]:
Test = test.copy()
cols=le.inverse_transform([*range(3)])
Test[cols]= test_predictions

In [ ]:
submit = Test[["Game_ID",'Away win', 'Draw', 'Home Win']]
submit.drop_duplicates(subset = ["Game_ID"], inplace=True)
submit = submit.reset_index(drop=True)
submit.to_csv("submission_lr.csv", index=False)